In [1]:
import numpy as np
import pandas as pd

import sys

from os.path import expanduser
sys.path.append(expanduser('~') + '/Lab/Utils/Python/')

from Conversions.translate import *
from Strings.is_a import *
from Math.normalize import z_transform_mode

# Initial setup

In [2]:
paper_pmid = 29377792
paper_name = 'farkas_pal_2018' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

In [5]:
path_to_genes = '../../Private-Utils/datasets_gene2.txt'
path_to_consensus_tested = '../../Private-Utils/yp_2020-09-01_orfs.txt'

# Load & process the data

In [15]:
original_data = pd.read_excel('raw_data/elife-29845-supp1-v1.xlsx', sheet_name='data')

In [16]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4971 x 122


In [17]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [18]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [19]:
original_data.loc[original_data['ORF']=='YLR287-A','ORF'] = 'YLR287C-A'

In [20]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [21]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

            ORF  gene  epsilon.mean  epsilon.pvalue  replicates.no.yEVenus  \
index_input                                                                  
0            WT  HIS3           NaN             NaN                 3371.0   

             mean.fitness.no.yEVenus  sd.fitness.no.yEVenus  \
index_input                                                   
0                           0.999996               0.034786   

             replicates.yEVenus  mean.fitness.yEVenus  sd.fitness.yEVenus  \
index_input                                                                 
0                        3376.0              0.940546            0.037521   

             ...  GO.0051603  GO.0051604  GO.0051726  GO.0055085  GO.0055086  \
index_input  ...                                                               
0            ...         NaN         NaN         NaN         NaN         NaN   

             GO.0061025  GO.0070271  GO.0070647  GO.0070925  GO.0071554  
index_input                 

In [22]:
original_data = original_data.loc[t,:]

In [23]:
original_data['data'] = original_data['epsilon.mean']

In [24]:
original_data.set_index('ORF', inplace=True)
original_data.index.name='orf'

In [25]:
original_data = original_data.groupby(original_data.index).mean()

In [26]:
original_data.shape

(4938, 120)

# Prepare the final dataset

In [27]:
data = original_data[['data']].copy()

In [28]:
dataset_ids = [16679]
datasets = datasets.reindex(index=dataset_ids)

In [29]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [30]:
data.head()

dataset_id,16679
data_type,value
orf,
YAL002W,-0.033532
YAL004W,0.001372
YAL005C,-0.009390
YAL007C,0.033624
YAL008W,0.027869


## Subset to the genes currently in SGD

In [31]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=original_data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [32]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [33]:
data.head()

,dataset_id,16679
,data_type,value
gene_id,orf,
2,YAL002W,-0.033532
1863,YAL004W,0.001372
4,YAL005C,-0.009390
5,YAL007C,0.033624
6,YAL008W,0.027869


# Normalize


In [34]:
def normalize_phenotypic_scores(df, has_tested=False):
    
    if not has_tested:
        
        genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
        genes = genes.reset_index().set_index('systematic_name', drop=False)
        
        yp_orfs = pd.read_csv(path_to_consensus_tested, header=None)
        yp_orfs = yp_orfs[1].values
        consensus_tested = [tuple(genes.loc[orf,['id','systematic_name']]) for orf in yp_orfs]
        
        df_index = [tuple(x) for x in df.index]
        
        consensus_tested = list(set(consensus_tested + df_index))
        
        df = df.reindex(index=consensus_tested, fill_value=0)
        
    df_norm = z_transform_mode(df)
    
    return df_norm

In [36]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [37]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [38]:
data_norm[data.isnull()] = np.nan

In [39]:
data_all = data.join(data_norm)

In [40]:
data_all.head()


dataset_id          16679          
data_type           value    valuez
gene_id orf                        
2       YAL002W -0.033532 -0.890926
1863    YAL004W  0.001372  0.234260
4       YAL005C -0.009390 -0.112669
5       YAL007C  0.033624  1.273953
6       YAL008W  0.027869  1.088459

# Print out

In [41]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [42]:
from IO.save_data_to_db3 import *

In [43]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 29377792...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.38s/it]
